In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read, write
from scipy import signal
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
from numpy.linalg import inv
import torch
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as I
from torch.nn import Sigmoid

from torch import transpose
from sklearn.neural_network import BernoulliRBM
import seaborn as sns
import warnings
warnings.simplefilter('ignore')
import numpy as np
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt

In [2]:
from helpers import *
#from nmf_utils import *
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Apply STFT :

## We can change :

* Window : Type of window
* nperseg : length of window
* noverlap : overlap between windows.
* nfft : fft length > window size

In [ ]:
!ffmpeg -i  "/content/drive/MyDrive/vocal_10.mp3" "vocal.wav"
!ffmpeg -i  "/content/drive/MyDrive/piano_10.mp3" "music.wav"

In [24]:

samplerate_m,music = read("music.wav")
music=music[:44100*1088,0]
length=music.shape[0]/samplerate_m
print('Shape of the music {}'.format(music.shape[0]))
print('Length : {:.2f}s'.format(length))
print('Sample rate : {}'.format(samplerate_m))


samplerate_s,speech = read("vocal.wav")
speech=speech[:44100*1088,0]
#speech = butter_lowpass_filter(speech,5000,44100)

length=speech.shape[0]/samplerate_s
print('Shape of the speech {}'.format(speech.shape[0]))
print('Length : {:.2f}s'.format(length))
print('Sample rate : {}'.format(samplerate_s))



Shape of the music 47980800
Length : 1088.00s
Sample rate : 44100
Shape of the speech 47980800
Length : 1088.00s
Sample rate : 44100


## Low band Filter

In [10]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y



## Resampling

In [25]:
from scipy import signal

In [26]:
rate = 44100/16000
speech = signal.resample(speech,int(speech.shape[0]/rate))
music=signal.resample(music,int(music.shape[0]/rate))
samplerate_m=int(samplerate_m/rate)
samplerate_s=samplerate_m
samplerate_t=samplerate_m

In [27]:
samplerate_t

16000

# Train / Validation

In [29]:
train_speech   = np.concatenate([speech[samplerate_t*60:samplerate_t*440],speech[samplerate_t*600:samplerate_t*900]],axis=0)
train_music    = np.concatenate([music[samplerate_t*60:samplerate_t*440],music[samplerate_t*600:samplerate_t*900]],axis=0)

test_speech ,  test_music =  speech[samplerate_t*440:samplerate_t*470]  , music[samplerate_t*440:samplerate_t*470]


In [30]:
SMR(train_speech,train_music)

SMR = 2.99


2.9862609974580523

In [31]:
#from helpers import *

def speech_nmf(Y,n_component,max_iter,a):


  Y[Y==0]=0.0001
  model = NMF(n_components=n_component,
            init='random',
            alpha=a,
            beta_loss='itakura-saito',
            solver="mu",
            max_iter=max_iter,
            random_state=0)
  B = model.fit_transform(Y)
  G = model.components_
  return B,G, model.reconstruction_err_
def music_nmf(Y,n_component,max_iter,a):


  Y[Y==0]=0.0001
  model = NMF(n_components=n_component,
            init='random',
            alpha=a,
            beta_loss='itakura-saito',
            solver="mu",
            max_iter=max_iter,
            random_state=0)
  B = model.fit_transform(Y)
  G = model.components_
  return B,G, model.reconstruction_err_




In [32]:
def Reconstruct(B,G,Ns,Nm,Yabs,p):
    
    numerators=[]
    B1=B[:,:Ns]
    B2=B[:,Ns:]
    G1=G[:Ns,:]
    G2=G[Ns:,:]
    
    
    numerators.append(np.power(np.matmul(B1,G1),p))
    numerators.append(np.power(np.matmul(B2,G2),p))

    denominator = np.power(np.matmul(B1,G1),p)+np.power(np.matmul(B2,G2),p)
  
    

    Sources=[]
    Masks=[]
    for i in range(2):

        Sources.append(np.multiply(numerators[i]/denominator,Yabs))
        Masks.append(numerators[i]/denominator)

    #print('Source shape = {}'.format(Sources[0].shape))
    
    return Sources,Masks
def training(music_s,
               speech_s,
               s_component = 128,
               m_component = 128,
               iterations = 200,
               s_alpha = 0,
               m_alpha = 0,
               p = 2,
             samplerate=16000,
                ):
  ###################################   STFT   #######################################
  WINDOW = 'hamming'
  WINDOW_SIZE=480
  OVERLAP = 0.6 * WINDOW_SIZE
  NFFT=512
  #_,speech_s,music_s = get_mixed_signal(spe)

  f,t,M= signal.stft(music_s,samplerate,window=WINDOW,nperseg=WINDOW_SIZE,noverlap=OVERLAP,nfft=NFFT)    # spectrum  of music signal

  f,t,S= signal.stft(speech_s,samplerate,window=WINDOW,nperseg=WINDOW_SIZE,noverlap=OVERLAP,nfft=NFFT)   # spectrum  of speech signal

  #####################################################################################


  ## Magnitude of each spectrum source 

  M_abs=np.abs(M)

  S_abs=np.abs(S)




  ##############################   NMF Training   #####################################
  print("Training Stage ....")
  B_speech,_,error_speech = speech_nmf(S_abs,s_component,iterations,s_alpha)
  B_music,_,error_music =   music_nmf(M_abs,m_component,iterations,m_alpha)
  print("Training finish ! ....")

  #####################################################################################

  B_mixed = np.concatenate([B_speech,B_music],axis = 1)                                # Concatenation of both  training dictionnaries
  print("SPEECH reconstruction Loss {}".format(error_speech))
  print("MUSIC reconstruction Loss {}".format(error_music))

  return B_mixed

In [33]:
def mixed_nmf(B_mixed,
              Y_test,
              iterations,
              a
              ):
  n_components=B_mixed.shape[1]
  model = NMF(n_components=n_components,
              init='random',
              alpha=a,
              beta_loss='itakura-saito',
              solver="mu",
              max_iter=iterations,
              random_state=0)
  
  model.fit(np.transpose(Y_test))

  model.components_ = np.transpose(B_mixed)

  G0 = model.transform(np.transpose(Y_test))

  return np.transpose(G0),model.reconstruction_err_,model.components_

In [34]:
def validation(B_mixed,
               speech,
               music,
               s_component,
               m_component,
               smr,
               iterations,
               samplerate=16000
               ):

  mixed_signal , speech_signal, music_signal = get_mixed_signal(speech,music,smr)
  #mixed_signal = mixed_signal
  WINDOW = 'hamming'
  WINDOW_SIZE=480
  OVERLAP = 0.6 * WINDOW_SIZE
  NFFT=512

  f,t,mixed_spectrum = signal.stft(mixed_signal,window=WINDOW,nperseg=WINDOW_SIZE,noverlap=OVERLAP,nfft=NFFT) # spectrum  of mixed signal

  mixed_abs = np.abs(mixed_spectrum)
  G_mixed,reconstruction,update_or_not = mixed_nmf(B_mixed,
              mixed_abs,
              iterations,
              0
              )
  #B_mixed,G_mixed,error_speech = speech_nmf(mixed_abs,s_component,iterations,0)
  print("ok")
  sources,masks = Reconstruct(B_mixed,
                              G_mixed,
                              s_component,
                              m_component,
                              mixed_spectrum,
                              2)
  _, speech_estimate =  signal.istft( sources[0],samplerate,window=WINDOW,nperseg=WINDOW_SIZE,noverlap=OVERLAP,nfft=NFFT)
  _, music_estimate =    signal.istft(sources[1],samplerate,window=WINDOW,nperseg=WINDOW_SIZE,noverlap=OVERLAP,nfft=NFFT)
  print("Separation finish ! ....")
  #print(speech_estimate.shape, speech.shape)
  sdr_speech = SDR(speech_estimate[:speech_signal.shape[0]],speech_signal)
  
  sdr_music = SDR(music_estimate[:music_signal.shape[0]],music_signal)
  print("Evaluation .....")
  #print(update_or_not)
  print("SMR ={} N_components ={}: \nSDR Speech = {}\nSDR Music  = {}".format(smr,s_component,sdr_speech,sdr_music))
  print(reconstruction)

  write("Speech{}.wav".format(iterations), samplerate, speech_estimate.astype(np.int16))
  write("Music.wav".format(iterations), samplerate, music_estimate.astype(np.int16))
  return smr,sdr_speech,sdr_music,update_or_not

In [39]:
B_all_list_2= []
for c in [2]:
  B_mixed = training(train_music,
               train_speech,
               s_component = c,
               m_component = c,
               iterations = 100,
               s_alpha = 0,
               m_alpha = 0,
               p = 2)
  B_mixed_new = MinMaxScaler().fit_transform(B_mixed)
  B_all_list_2.append(B_mixed_new)



Training Stage ....
Training finish ! ....
SPEECH reconstruction Loss 4462.14015906347
MUSIC reconstruction Loss 3140.7534270949936


In [41]:
for it in [1,10,100]:
  for i in range(len(B_all_list_2)):
    Q = np.int(B_all_list_2[i].shape[1] / 2 )
    for s in [0]:
      a,x,v,xxx = validation(B_all_list_2[i],test_speech,
                    test_music,
                    smr=s,
                    iterations=it,
                    m_component=Q,
                    s_component=Q)

ok
Separation finish ! ....
Evaluation .....
SMR =0 N_components =2: 
SDR Speech = 2.9348455449606248
SDR Music  = 0.4769865169695596
1022.1629455308959
ok
Separation finish ! ....
Evaluation .....
SMR =0 N_components =2: 
SDR Speech = 3.2005014293463327
SDR Music  = 0.7426424013552673
841.1128885143034
ok
Separation finish ! ....
Evaluation .....
SMR =0 N_components =2: 
SDR Speech = 3.5896611298929466
SDR Music  = 1.1318021019018816
738.8676148861664
